<a href="https://colab.research.google.com/github/Arenasaurio/explanaible-steam-nlp/blob/ArisaKeyboard-patch-2/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [3]:
df = pd.read_parquet("/content/drive/MyDrive/Datos_Modelo.parquet")

In [4]:
print(f"Forma inicial del DataFrame: {df.shape}")
if not df.empty and 'review_score' in df.columns:
    print(f"nulos en 'review score' antes de limpiar:\n{df['review_score'].isna().sum()}")
else:
    print("DataFrame is empty or 'review_score' column not found.")

Forma inicial del DataFrame: (2201192, 2)
nulos en 'review score' antes de limpiar:
0


In [5]:
df_limpio = df.dropna(subset=['review_score'])


In [6]:
df_limpio['review_score'] = df_limpio['review_score'].astype(int)

In [7]:
print(f"Forma del DataFrame limpio: {df_limpio.shape}")
print(f"Distribución de etiquetas después de la limpieza:\n{df_limpio['review_score'].value_counts()}")

Forma del DataFrame limpio: (2201192, 2)
Distribución de etiquetas después de la limpieza:
review_score
 1    1100596
-1    1100596
Name: count, dtype: int64


In [8]:
#extraer listas
reviews = df_limpio['review_text'].tolist()
labels = df_limpio['review_score'].tolist()

In [9]:
#mapear etiquetas a 1 y 0
y_mapped = np.array([0 if x == -1 else 1 for x in labels])

In [10]:
#cargar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#generar embeddings
embeddings = model.encode(reviews, show_progress_bar=True)

Batches:   0%|          | 0/68788 [00:00<?, ?it/s]

In [12]:
#guardar
np.save('embeddings.npy', embeddings)
np.save('labels_mapped.npy', y_mapped)

In [13]:
print(f"Forma del array de embeddings (X): {embeddings.shape}")
print(f"Forma del array de etiquetas (y): {y_mapped.shape}")


Forma del array de embeddings (X): (2201192, 384)
Forma del array de etiquetas (y): (2201192,)


In [14]:
embeddingscopy = embeddings

In [15]:
display(embeddings[:1])

array([[-2.18965188e-02,  2.47201081e-02, -2.87570395e-02,
        -7.31733292e-02,  2.68594343e-02,  1.69045925e-02,
         7.35135451e-02,  7.90307745e-02, -2.45406181e-02,
         7.87125379e-02, -1.51085034e-02, -2.97033694e-02,
         4.36032899e-02, -1.36522166e-02,  6.06360808e-02,
        -8.00720043e-03,  1.03937887e-01, -1.15647182e-01,
         3.13400989e-03, -1.18471393e-02, -3.81409936e-02,
        -3.67038622e-02,  2.09907815e-02, -1.68212689e-02,
         1.79156940e-02,  3.51764038e-02, -7.30648218e-03,
         9.99435112e-02, -1.02385864e-01, -4.19805385e-02,
         3.98927331e-02,  6.33708164e-02, -1.62096359e-02,
         4.64103222e-02,  9.51347407e-03,  2.03598049e-02,
         3.30239162e-02, -4.04623188e-02, -2.66862772e-02,
        -8.22296962e-02, -9.26577896e-02, -1.72809437e-02,
         4.34691049e-02, -1.80760305e-02,  1.43802874e-02,
         4.40071337e-02,  4.72963415e-02, -1.82275828e-02,
        -5.29839508e-02, -2.51180166e-03,  6.86460882e-0

In [6]:
df.tail(20)

,review_text,review_score
2201172,Reminds me of the games I played in elementary...,-1
2201173,I dont like this game,-1
2201174,The actual game play of Puzzle Pirates is grea...,-1
2201175,"Rating based on current state of play, as per ...",-1
2201176,This is just appalling.,-1
2201177,Set my age as less than 5 by mistake. Apparent...,-1
2201178,It is terrible because I cant ge on because of...,-1
2201179,"Was fun for the first 30 minutes or so, got bo...",-1
2201180,The game is very awefull and strange. I think ...,-1
2201181,"A very good game, got sick of it after a while...",-1
